Mutually Friend Numbers usando OpenMP

João Augusto Leite 743551 \\
Vinicius Henrique Carvalho \\
Daniel Lamounier Heringer 743524

In [3]:
%%writefile arquivo.c
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <omp.h>
#include <sys/time.h>

int gcd(int u, int v) { // MDC
	if (v == 0)
		return u;
	return gcd(v, u % v);
}

void friendly_numbers(long int start, long int end) {
	long int last = end - start + 1;

	long int *the_num;
	the_num = (long int*) malloc(sizeof(long int) * last); //armazena todos os numeros da cadeia
	long int *num;
	num = (long int*) malloc(sizeof(long int) * last);
	long int *den;
	den = (long int*) malloc(sizeof(long int) * last);

	long int i, j, factor, ii;
	long int sum, done, n;

  /*
  Essa diretiva indica ao OpenMP que iremos parelelizar este for usando o
  escalonador no modo dynamic, que atribuirá as tarefas em tempo de execução.
  Isso é mais eficiente nesse caso pois um número pode demorar muito mais do que
  outro para calcular os divisores. Com dynamic, o escalonador pode gerenciar
  isso e impedir que alguma thread fique ociosa.
  Além disso, definimos explicitamente as variáveis shared, que serão acessadas
  por todas as threads, e as variaveis privadas, que cada thread possuirá
  uma cópia do valor. Definimos os vetores como shared e as variáveis de controle
  como privadas.
  */
  #pragma omp parallel for schedule(dynamic) shared(the_num, num, den) private(i, ii, sum, done, n, factor)
	for (i = start; i <= end; i++) {
		ii = i - start;
    //printf("Thread id: %d using index %ld\n", omp_get_thread_num(), ii);
		sum = 1 + i;
		the_num[ii] = i;
		done = i;
		factor = 2;
		while (factor < (done/2)+1) {  // faz a soma dos elementos divisiveis por i
			if ((i % factor) == 0) {
				sum += (factor);
			}
			factor++;
		}

		num[ii] = sum; // recebe a soma do elemento the_num[ii]
		den[ii] = i; // proprio numero
		n = gcd(num[ii], den[ii]); // mdc
		num[ii] /= n; 
		den[ii] /= n;
		/*
			num[ii]/den[ii]
		*/
	} // end for

	for (i = 0; i < last; i++) {
		for (j = i + 1; j < last; j++) {
			if ((num[i] == num[j]) && (den[i] == den[j]))
				printf("%ld and %ld are FRIENDLY\n", the_num[i], the_num[j]); // printa numeros mutuamente amigaveis
		}
	}

	free(the_num);
	free(num);
	free(den);
}

int main(int argc, char **argv) {
    
	struct timeval inic,fim;
	long int start;
	long int end;

	while (1) {
		scanf("%ld %ld", &start, &end);
		if (start == 0 && end == 0)
			break;
		printf("Number %ld to %ld\n", start, end);
	  gettimeofday(&inic,0);
		friendly_numbers(start, end);
	  gettimeofday(&fim,0);
    printf("\nElapsed time:%f sec\n", (fim.tv_sec+fim.tv_usec/1000000.) - (inic.tv_sec+inic.tv_usec/1000000.));
	}

	return EXIT_SUCCESS;
}


Writing arquivo.c


In [4]:
%%time
!gcc arquivo.c -o out -fopenmp
!./out

1 15000
Number 1 to 15000
1 and 6 are FRIENDLY
1 and 28 are FRIENDLY
1 and 496 are FRIENDLY
1 and 8128 are FRIENDLY
6 and 28 are FRIENDLY
6 and 496 are FRIENDLY
6 and 8128 are FRIENDLY
12 and 234 are FRIENDLY
28 and 496 are FRIENDLY
28 and 8128 are FRIENDLY
30 and 140 are FRIENDLY
30 and 2480 are FRIENDLY
30 and 6200 are FRIENDLY
40 and 224 are FRIENDLY
42 and 3472 are FRIENDLY
56 and 3724 are FRIENDLY
60 and 1170 are FRIENDLY
66 and 308 are FRIENDLY
66 and 5456 are FRIENDLY
78 and 364 are FRIENDLY
78 and 6448 are FRIENDLY
80 and 200 are FRIENDLY
84 and 270 are FRIENDLY
84 and 1488 are FRIENDLY
84 and 1638 are FRIENDLY
102 and 476 are FRIENDLY
102 and 8432 are FRIENDLY
114 and 532 are FRIENDLY
114 and 9424 are FRIENDLY
120 and 672 are FRIENDLY
132 and 2574 are FRIENDLY
135 and 819 are FRIENDLY
138 and 644 are FRIENDLY
138 and 11408 are FRIENDLY
140 and 2480 are FRIENDLY
140 and 6200 are FRIENDLY
150 and 700 are FRIENDLY
150 and 12400 are FRIENDLY
168 and 11172 are FRIENDLY
174 and 812 